In [106]:
from openai import OpenAI

In [107]:
from dotenv import load_dotenv

In [108]:
from supabase import create_client, Client

In [109]:
import anthropic

In [110]:
load_dotenv()

True

In [111]:
import os

In [112]:
import requests

In [114]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [115]:
anthropic_api_key = os.getenv("CLAUDE_API_KEY")

In [136]:
client_anthropic = anthropic.Anthropic(api_key = anthropic_api_key)

In [264]:
import json

In [22]:
supabase_url = os.environ.get("SUPABASE_URL")

In [23]:
supabase_api_key = os.environ.get("SUPBASE_KEY")

In [25]:
supabase: Client = create_client(supabase_url, supabase_api_key)

In [27]:
sql_query

'\nSELECT * FROM webhooks order by created_at ASC\n'

In [283]:
result = supabase.table('webhooks')\
    .select('id, news_output,source_urls, news_date')\
    .order('created_at', desc=False)\
    .execute()

result.data

[{'id': 2,
  'news_output': 'Reuters (Jan 24, 2026) reports the Singapore government will invest over S$1 billion (about $778.8 million) in public AI research through 2030. The funding targets priority research areas including responsible and resource-efficient AI, talent development from pre-university to faculty levels, and building capabilities to support industry adoption of AI. The announcement builds on prior Singapore commitments (e.g., 2024 HPC funding and AI Singapore investments) and is intended to strengthen national AI capabilities and global competitiveness.',
  'source_urls': ['https://www.reuters.com/world/asia-pacific/singapore-invest-over-779-million-public-ai-research-through-2030-2026-01-24/'],
  'news_date': '2026-01-24'},
 {'id': 3,
  'news_output': "TechCrunch (Jan 24, 2026) published an analysis titled 'A new test for AI labs: Are you even trying to make money?' The piece proposes a five-level scale to rate AI labs by commercial ambition and summarizes recent fun

In [301]:
def prompt_generator_research():
    formatted_prompt = []
    
    for item in result.data:
        event_id = f"{item['id']}_{item['news_date']}"
        prompt = f"""You are given news summary: {item['news_output']} and source: {item['source_urls'][0]}, you are supposed to run a deep research and read other sources as well. And then give a well structured ouput in the form of points and sources."""

        prompt_dict = {
            'event_id': event_id,
            'prompt': prompt
        }
        
        formatted_prompt.append(prompt_dict)

    return [formatted_prompt[1]]

In [302]:
prompt_generator_research()

[{'event_id': '3_2026-01-24',
  'prompt': "You are given news summary: TechCrunch (Jan 24, 2026) published an analysis titled 'A new test for AI labs: Are you even trying to make money?' The piece proposes a five-level scale to rate AI labs by commercial ambition and summarizes recent funding and personnel developments (references Humans&'s large seed round and leadership departures at Thinking Machines Lab), arguing these developments show how investor appetite and commercialization strategies are shaping which labs pursue productization versus pure research. and source: https://techcrunch.com/2026/01/24/a-new-test-for-ai-labs-are-you-even-trying-to-make-money/, you are supposed to run a deep research and read other sources as well. And then give a well structured ouput in the form of points and sources."}]

In [303]:
system_prompt = prompt_generator_research()

In [306]:
def openai_research():
    research_output = []
    for i in system_prompt:
        
        response = client.responses.create(
            model = "gpt-4o",
            tools = [{
                "type": "web_search"
            }],

            include = ["web_search_call.action.sources"],
            input = i['prompt'],
        )

        output = response.output_text

        final_dict = {
            'event_id': i['event_id'],
            'output': output
        }

        research_output.append(final_dict)

    return research_output

In [307]:
openai_research()

[{'event_id': '3_2026-01-24',
  'output': "Below is a structured, in‑depth analysis (approx. 800+ words) of the TechCrunch piece “A new test for AI labs: Are you even trying to make money?” (published January\u202f24,\u202f2026) along with complementary context drawn from other recent journalistic sources. Each statement includes citations to support the analysis.\n\n---\n\n## 1. The Five‑Level Scale: Measuring Commercial Ambition in AI Labs\n\n- On **January\u202f24,\u202f2026**, TechCrunch editor Russell\u202fBrandom introduced a **five‑level scale** to assess AI labs not by current revenue, but by **commercial ambition**. The levels range from **Level\u202f1** (“True wealth is when you love yourself”) to **Level\u202f5** (“We are already making millions of dollars every day, thank you very much”) ([techcrunch.com](https://techcrunch.com/2026/01/24/a-new-test-for-ai-labs-are-you-even-trying-to-make-money/)).\n\n- The scale’s key innovation lies in separating **ambition** from **succe

In [316]:
def openai_article_builder(researched_data):
    final_news_article = []
    for i in researched_data:

        output = i['output']

        event_id = i['event_id']
        print(event_id)
        
        response = client.responses.create(
            model = "gpt-4.1",
            input = f""" 
            You are a professional tech journalist for "The AI Times" newspaper.

            Your task: Write a comprehensive news article covering today's AI developments.
            
            Entire news story and its details: {output}
            
            INSTRUCTIONS:
            1. Write a newspaper-quality article (800-1000 words)
            2. Create an engaging headline
            3. Use journalistic style: objective, clear, professional and opinionated
            4. Include all the sources & links at the end and not ANY sources or links during the article (THIS IS NON NEGOTIABLE)

            You are ONLY supposed to refer to the sources and links and NOT supposed to hallucinate. 
            
            However, the opinion you have be your own but it should be based on the above article. 
            """
        )


        final_output = response.output_text
        print(final_output)

        json_format = json_formatter(final_output, "openai", event_id)
        print(json_format)

        final_news_article.append(json_format)

    return final_news_article

In [317]:
b = [{'event_id': '3_2026-01-24',
  'output': "Below is a structured, in‑depth analysis (approx. 800+ words) of the TechCrunch piece “A new test for AI labs: Are you even trying to make money?” (published January\u202f24,\u202f2026) along with complementary context drawn from other recent journalistic sources. Each statement includes citations to support the analysis.\n\n---\n\n## 1. The Five‑Level Scale: Measuring Commercial Ambition in AI Labs\n\n- On **January\u202f24,\u202f2026**, TechCrunch editor Russell\u202fBrandom introduced a **five‑level scale** to assess AI labs not by current revenue, but by **commercial ambition**. The levels range from **Level\u202f1** (“True wealth is when you love yourself”) to **Level\u202f5** (“We are already making millions of dollars every day, thank you very much”) ([techcrunch.com](https://techcrunch.com/2026/01/24/a-new-test-for-ai-labs-are-you-even-trying-to-make-money/)).\n\n- The scale’s key innovation lies in separating **ambition** from **success**; a lab may not yet have product revenue, but its positioning on the scale reflects whether it is striving for commercialization (Levels\u202f3–5) or leaning toward pure research (Levels\u202f1–2) ([techcrunch.com](https://techcrunch.com/2026/01/24/a-new-test-for-ai-labs-are-you-even-trying-to-make-money/)).\n\n---\n\n## 2. Placement of Notable Labs on the Scale\n\n### 2.1 Humans&\n\n- **Humans&**, founded by former Anthropic, xAI, and Google researchers, recently raised **$480 million in seed funding** at a **$4.48 billion valuation**. Prominent investors include Nvidia, Jeff Bezos, GV, SV Angel, and Emerson Collective ([techcrunch.com](https://techcrunch.com/2026/01/20/humans-a-human-centric-ai-startup-founded-by-anthropic-xai-google-alums-raised-480m-seed-round/?utm_source=openai)).\n\n- Their stated ambition is to build **AI workplace tools**—essentially evolving platforms like Slack, Jira, and Google Docs—but the company remains vague about specifics. This ambiguity places them at **Level\u202f3** on the ambition scale: promising product ideas but lacking clear execution plans ([techcrunch.com](https://techcrunch.com/2026/01/24/a-new-test-for-ai-labs-are-you-even-trying-to-make-money/)).\n\n### 2.2 Thinking Machines Lab\n\n- **Thinking Machines Lab** (TML), co‑founded in early 2025 by ex‑OpenAI CTO Mira\u202fMurati and others, raised a **$2\u202fbillion seed round** at a **$12\u202fbillion valuation** ([fortune.com](https://fortune.com/2025/02/18/former-openai-cto-mira-murati-finally-unveils-her-thinking-machines-lab-startup-and-a-leadership-team-stacked-with-former-openai-colleagues/?queryly=related_article&utm_source=openai)).\n\n- At first glance, TML projected as a **Level\u202f4** company: it appeared to have a “detailed multi‑stage plan to become the richest human beings on Earth” ([techcrunch.com](https://techcrunch.com/2026/01/24/a-new-test-for-ai-labs-are-you-even-trying-to-make-money/)). However, a wave of **high‑profile departures**—especially CTO Barret\u202fZoph, co‑founder Luke\u202fMetz, researcher Sam\u202fSchoenholz, plus others—has cast doubt on that level of ambition ([techcrunch.com](https://techcrunch.com/2026/01/24/a-new-test-for-ai-labs-are-you-even-trying-to-make-money/)).\n\n- Brandom concludes that while TML might have wanted to be Level\u202f4, recent instability suggests it may in reality be at **Level\u202f2 or 3**, or even be **sliding downward** ([techcrunch.com](https://techcrunch.com/2026/01/24/a-new-test-for-ai-labs-are-you-even-trying-to-make-money/)).\n\n### 2.3 World Labs & Safe Superintelligence (SSI)\n\n- **World Labs**, co‑founded by Fei‑Fei\u202fLi, raised **$230\u202fmillion in 2024** and has delivered both a world‑generating model and a commercial product. Observers suggest it has quickly ascended to **Level\u202f4**, with the potential for **Level\u202f5** in the near future ([techcrunch.com](https://techcrunch.com/2026/01/24/a-new-test-for-ai-labs-are-you-even-trying-to-make-money/)).\n\n- **Safe Superintelligence (SSI)**, founded by Ilya\u202fSutskever, appears firmly at **Level\u202f1**—a research‑only orientation, with even a turned down Meta acquisition and no products yet ([techcrunch.com](https://techcrunch.com/2026/01/24/a-new-test-for-ai-labs-are-you-even-trying-to-make-money/)).\n\n---\n\n## 3. Broader Context: Talent Flows and Commercial Pressure in AI Startups\n\n### 3.1 Thinking Machines Lab: Talent Exodus\n\n- **Barret\u202fZoph**, TML's CTO and co‑founder, was **removed** or **parted ways** with the company in mid‑January 2026. He promptly rejoined OpenAI, along with Luke\u202fMetz and Sam\u202fSchoenholz ([nypost.com](https://nypost.com/2026/01/21/business/co-founder-of-12b-ai-startup-fired-after-boss-learns-about-office-romance-report/?utm_source=openai)).\n\n- Some reports hint at an “unethical conduct” allegation against Zoph, including possibly leaking confidential info—though OpenAI publicly welcomed him, suggesting either the concerns were overblown or overshadowed by his technical value ([wired.com](https://www.wired.com/story/thinking-machines-lab-cofounders-leave-for-openai?utm_source=openai)).\n\n- Additional staff, such as Lia\u202fGuy, also departed to rejoin OpenAI, while others like Ian\u202fO’Connell left for unknown destinations—amplifying concerns about internal instability at TML ([theinformation.com](https://www.theinformation.com/briefings/two-ai-staffers-depart-muratis-thinking-machines/?utm_source=openai)).\n\n### 3.2 Strategic Implications\n\n- These exits occur at a critical moment: TML reportedly was pursuing a **$4–5\u202fbillion funding round** aimed at a **$50\u202fbillion valuation**, making it especially vulnerable to disruption ([theinformation.com](https://www.theinformation.com/briefings/two-ai-staffers-depart-muratis-thinking-machines/?utm_source=openai)).\n\n- Other analysis highlights that despite vast funding, TML may be hampered by lacking infrastructure (“compute gravity well”) required for frontier AI research, offering OpenAI an edge in reclaiming talent who seek immediate experimental environments ([creati.ai](https://creati.ai/ai-news/2026-01-17/thinking-machines-lab-loses-founders-to-openai/?utm_source=openai)).\n\n- **Soumith\u202fChintala**, co‑creator of PyTorch, has stepped in as the new CTO—a move that's intended to restore confidence, signaling a shift toward open‑source, community‑centric tooling rather than proprietary model development ([businessinsider.com](https://www.businessinsider.com/meta-soumith-chintala-mira-murati-thinking-machines-lab-pytorch-ai-2025-11?utm_source=openai)).\n\n---\n\n## 4. Synthesis: What This Means for the AI Lab Landscape\n\n### 4.1 Investment vs. Execution\n\n- Brandom’s scale underscores that vast capital and prestige **do not guarantee commercial drive**. Without tangible productization strategy, labs remain ambiguous in their ambitions ([techcrunch.com](https://techcrunch.com/2026/01/24/a-new-test-for-ai-labs-are-you-even-trying-to-make-money/)).\n\n- **Humans&** illustrates early‑stage promise: strong funding and intent to build workplace tools—but precise product plans are undefined, placing it at Level\u202f3 ([techcrunch.com](https://techcrunch.com/2026/01/20/humans-a-human-centric-ai-startup-founded-by-anthropic-xai-google-alums-raised-480m-seed-round/?utm_source=openai)).\n\n- **Thinking Machines Lab** initially appeared at Level\u202f4, but **internal turmoil and leadership churn** have prompted reevaluation toward Level\u202f2–3 ([techcrunch.com](https://techcrunch.com/2026/01/24/a-new-test-for-ai-labs-are-you-even-trying-to-make-money/)).\n\n### 4.2 The Talent War & Strategic Vulnerability\n\n- The swift **return of key personnel to OpenAI** reflects how **compute infrastructure**, organizational stability, and institutional prestige weigh heavily on researchers’ decisions—not merely startup equity or mission ([creati.ai](https://creati.ai/ai-news/2026-01-17/thinking-machines-lab-loses-founders-to-openai/?utm_source=openai)).\n\n- It's a warning for well‑funded AI startups: without organizational cohesion and infrastructure, even splashy ventures may lose key talent.\n\n### 4.3 Potential Repositioning for TML\n\n- The appointment of **Soumith\u202fChintala** could mark a strategic pivot: embracing open‑source tooling and developer ecosystems may rebuild credibility and carve a more resilient niche.\n\n- This pivot may redefine TML’s ambition—not as a closed‑doors model developer, but as a platform‑oriented, collaborative infrastructure player.\n\n---\n\n## 5. Conclusion & Outlook\n\n- **TechCrunch’s five‑level scale** (Levels 1 to 5) offers a valuable lens to evaluate AI labs on **ambition**, decoupled from actual revenue ([techcrunch.com](https://techcrunch.com/2026/01/24/a-new-test-for-ai-labs-are-you-even-trying-to-make-money/)).\n\n- **Humans&** stands at Level\u202f3: big ideas and funding, but unclear execution.\n\n- **Thinking Machines Lab** initially seemed Level\u202f4, but recent **executive departures** and structural shake‑ups push it toward Level\u202f2–3.\n\n- **World Labs** may be ascending to Level\u202f4, whereas **SSI** remains Level\u202f1.\n\n- Across the AI ecosystem, **talent retention and alignment of commercial strategy** are becoming central to a lab’s trajectory—no amount of funding can substitute for cohesive leadership, infrastructure readiness, and credible path to productization.\n\n---\n\nIf you'd like, I can further analyze individual labs’ trajectories, compare funding timelines, or explore how investor expectations shift in this evolving commercial‑research landscape."}]

openai_article_builder(b)

3_2026-01-24
**Headline:**  
**Beyond Billion-Dollar Hype: Are AI Labs Chasing Profit or Just Prestige?**

**By [AI Times Correspondent]**

In the feverish world of artificial intelligence, where billion-dollar valuations and monumental fundraising rounds have become almost pedestrian, the question haunting the industry is neither “who’s next?” nor “how much will they raise?” but, as posed by TechCrunch’s Russell Brandom, “Are you even trying to make money?” On January 24, 2026, Brandom’s five-level “commercial ambition” scale upended the typical focus on valuation and technical prowess, shifting the spotlight onto a more fundamental but often ignored question: after the money and the models, do these AI labs actually intend to ship products — and get paid?

### The Five-Level Test: Ambition over Revenue

Brandom’s five-level framework is not a measure of balance sheets or product-market fit. Rather, it grades labs based on the visible intent to pursue commercialization. At Level 1 sit

[[{'model': 'openai',
   'headline': 'Beyond Billion-Dollar Hype: Are AI Labs Chasing Profit or Just Prestige?',
   'content': "In the feverish world of artificial intelligence, where billion-dollar valuations and monumental fundraising rounds have become almost pedestrian, the question haunting the industry is neither “who’s next?” nor “how much will they raise?” but, as posed by TechCrunch’s Russell Brandom, “Are you even trying to make money?” On January 24, 2026, Brandom’s five-level “commercial ambition” scale upended the typical focus on valuation and technical prowess, shifting the spotlight onto a more fundamental but often ignored question: after the money and the models, do these AI labs actually intend to ship products — and get paid?\n\n### The Five-Level Test: Ambition over Revenue\n\nBrandom’s five-level framework is not a measure of balance sheets or product-market fit. Rather, it grades labs based on the visible intent to pursue commercialization. At Level 1 sit pure re

In [331]:
def anthropic_research():
    research_ouput_anthropic = []
    for i in system_prompt:
        response = client_anthropic.messages.create(
            model = "claude-haiku-4-5-20251001",
            max_tokens = 4096,
            messages = [
                {
                    "role": "user",
                    "content": f"{i['prompt']}"
                }
            ],
            tools = [
                {
                    "type": "web_search_20250305",
                    "name": "web_search",
                    "max_uses": 5
                }
            ]
        )

        final_dict = {
            'event_id': i['event_id'],
            'blocks': response.content
        }

        research_ouput_anthropic.append(final_dict)

    return research_ouput_anthropic

In [332]:
def extract_final_text(blocks):
    last_tool_idx = -1
    for i,b in enumerate(blocks):
        if getattr(b, "type", None) in ("server_tool_use", "web_search_tool_result"):
            last_tool_idx = i

    text_parts = []
    for b in blocks[last_tool_idx + 1:]:
        if getattr(b, "type", None) == "text":
            text_parts.append(b.text)

    return "\n".join(text_parts).strip()

In [333]:
def extract_sources(blocks):
    sources = []

    for b in blocks:
        if getattr(b, "type", None) == "web_search_tool_result":
            for r in getattr(b, "results", []):
                sources.append({
                    "title": r.get("title"),
                    "url": r.get("url"),
                    "source": r.get("source")
                })

    return sources

In [336]:
def final_claude_responses():
    results = anthropic_research()
    
    claude_responses = []
    
    for item in results:
        event_id = item['event_id']
        blocks = item['blocks']
        
        story = extract_final_text(blocks)          
        sources = extract_sources(blocks)           

        
        sources_text = "\n".join(
            f"- {s.get('title', '(no title)')}\n  {s.get('url', '')}"
            for s in sources
            if s.get("url")
        ).strip()

        
        payload = f"{story}".strip()

        if sources_text:
            payload += "\n\nSOURCES:\n" + sources_text

    
        if payload.strip():
            claude_responses.append({
                "event_id": event_id,
                "output": payload
            })

    return claude_responses

In [337]:
final_claude_responses()

[{'event_id': '3_2026-01-24',
  'output': 'Based on my comprehensive research, here is a well-structured analysis of the TechCrunch article and related AI industry developments:\n\n## **COMPREHENSIVE ANALYSIS: AI LABS\' COMMERCIAL AMBITION & INDUSTRY TRENDS**\n\n---\n\n### **I. THE CORE ARTICLE THESIS**\n\n\n2026 marks a shift where AI moves from hype to pragmatism, with focus shifting from building ever-larger language models toward making AI usable\n. \nKey AI players are prioritizing productization—the process of turning AI models into practical, user-friendly tools for both businesses and individual consumers\n.\n\n---\n\n### **II. MEGA-FUNDING ROUNDS: THE NEW NORMAL FOR AI STARTUPS**\n\n#### **A. Humans& - The Prime Example of Capitalization Without Product**\n\n\nHumans&, with a philosophy that AI should empower people rather than replace them, raised $480 million in seed funding at a $4.48 billion valuation, with investors including chipmaker Nvidia, Amazon founder Jeff Bezos, a

In [340]:
def claude_article_builder(researched_data):
    final_news_article = []
    for i in researched_data:
        response = client_anthropic.messages.create(
            model = "claude-sonnet-4-5",
            max_tokens = 1800,
            messages = [{
                "role": "user",
                "content":  f"""
                You are a professional tech journalist for "The AI Times" newspaper.
                
                Your task: Write a comprehensive news article covering today's AI developments.
                Entire news story and its details: {i['output']}
                
                INSTRUCTIONS:
                1. Write a newspaper-quality article (800-1000 words)
                2. Create an engaging headline
                3. Use journalistic style: objective, clear, professional and opinionated
                4. Include all the sources & links at the end and not ANY sources or links during the article (THIS IS NON NEGOTIABLE)
                
                You are ONLY supposed to refer to the sources and links and NOT supposed to hallucinate. 
                
                However, the opinion you have be your own but it should be based on the above article. 
            """
            }]
        )

        event_id = i['event_id']

        final_output = response.content[0].text

        json_format = json_formatter(final_output, "claude",event_id)

        final_news_article.append(json_format)

    return final_news_article

In [339]:
a = [{'event_id': '3_2026-01-24',
  'output': 'Based on my comprehensive research, here is a well-structured analysis of the TechCrunch article and related AI industry developments:\n\n## **COMPREHENSIVE ANALYSIS: AI LABS\' COMMERCIAL AMBITION & INDUSTRY TRENDS**\n\n---\n\n### **I. THE CORE ARTICLE THESIS**\n\n\n2026 marks a shift where AI moves from hype to pragmatism, with focus shifting from building ever-larger language models toward making AI usable\n. \nKey AI players are prioritizing productization—the process of turning AI models into practical, user-friendly tools for both businesses and individual consumers\n.\n\n---\n\n### **II. MEGA-FUNDING ROUNDS: THE NEW NORMAL FOR AI STARTUPS**\n\n#### **A. Humans& - The Prime Example of Capitalization Without Product**\n\n\nHumans&, with a philosophy that AI should empower people rather than replace them, raised $480 million in seed funding at a $4.48 billion valuation, with investors including chipmaker Nvidia, Amazon founder Jeff Bezos, and VC firms SV Angel, GV, and Laurene Powell Jobs\' firm Emerson Collective\n.\n\n**Key Context:**\n- \nFounders include Andi Peng, a former Anthropic researcher who worked on reinforcement learning and post-training of Claude 3.5 through 4.5; Georges Harik, Google\'s seventh employee; Eric Zelikman and Yuchen He, two former xAI researchers who helped develop Grok; and Noah Goodman, a Stanford professor\n\n- \nThe company plans to launch its first product early this year\n\n- \nThe round values the company at $4.48 billion despite being roughly three months old and employing around 20 people—about $24 million per employee, which some view as either an inspiring bet on human potential or a sign that "seed stage" has lost all meaning\n\n\n#### **B. Historical Context: Thinking Machines Lab\'s Cautionary Tale**\n\n\nThe largest seed round in history belongs to Thinking Machines Lab, which raised $2 billion last July at a $12 billion valuation, led by Andreessen Horowitz. Founded by former OpenAI CTO Mira Murati alongside top researchers from Meta and Google, the company initially attracted enormous enthusiasm, though the departure of half of the company\'s founding team across recent months suggests that massive capital and pedigree don\'t guarantee immediate success\n.\n\n**Leadership Crisis Details:**\n- \nTwo co-founders, Barret Zoph and Luke Metz, are heading back to OpenAI, alongside Sam Schoenholz, another former OpenAI staffer who had joined the startup. This reversal comes just six months after the company secured a record-breaking $2 billion funding round\n\n- \nThe company has also lost other key personnel, including co-founder Andrew Tulloch, who left to join Meta in October\n\n- \nThe departures may signal that the defectors are interested in building products and more applied AI research than they were able to do at Thinking Machines. Other neo labs may also be struggling to turn research into products and establish a business model\n\n\n---\n\n### **III. THE PRODUCTIZATION VS. PURE RESEARCH DIVIDE**\n\n\nExperts see 2026 as a year of transition, one that evolves from brute-force scaling to researching new architectures, from flashy demos to targeted deployments, and from agents that promise autonomy to ones that actually augment how people work\n.\n\n#### **A. Enterprise AI Adoption Pressures**\n\n\nTechCrunch surveyed 24 enterprise-focused VCs, and they overwhelmingly think 2026 will be the year when enterprises start to meaningfully adopt AI, see value from it, and increase their budgets for the tech—though enterprise VCs have been saying that for three years now\n.\n\n\nA boon for AI startups in 2026 will be the transition of enterprises who tried to build in-house solutions and have now realized the difficulty and complexity required in production at scale\n.\n\n#### **B. Real-World Examples of Productization Success**\n\n\nLMArena launched as a commercial venture with a $100 million seed round in May at a $600 million valuation. In September, it publicly launched a commercial service, AI Evaluations. This gave LMArena an annualized consumption rate of $30 million as of December, less than four months after launch\n.\n\n\nConverge Bio has completed over 40 programs with more than a dozen pharmaceutical and biotech customers. The team has grown rapidly, increasing to 34 employees from just nine in November 2024\n.\n\n---\n\n### **IV. INVESTMENT TRENDS: PEDIGREE-DRIVEN FUNDING**\n\n\nThe megadeals for young companies follow a trend of investors throwing money at startups founded by breakaways of major AI labs\n.\n\n\nIn 2026, AI funding is increasingly less about what you\'ve built and more about where you escaped from\n.\n\n**Broader Investment Patterns:**\n- \nNvidia made its first direct investment in Anthropic, committing up to $10 billion as part of a strategic round that included a $5 billion check from Microsoft\n\n- \nAccording to its own projections, Anthropic will burn through close to $20 billion before it becomes profitable. OpenAI will need roughly $150 billion before it starts generating cash. Companies with such staggering capital needs have no choice but to tap public capital markets\n\n\n---\n\n### **V. 2026 INDUSTRY OUTLOOK: KEY PREDICTIONS**\n\n#### **A. Shift to Enterprise Deployment**\n\n\n2026 marks the pivot from experimental AI to broad enterprise deployment. AI is integrating into core business workflows rather than functioning as standalone tools\n.\n\n#### **B. Productization as Competitive Moat**\n\n\nCompanies like Anthropic, Google, OpenAI, and Microsoft are all doubling down on developing platforms that reach real users and solve actual workplace challenges. This marks a departure from the past focus on pure technological innovation or academic breakthroughs\n.\n\n\nThe AI narrative has mostly been dominated by model performance on key industry benchmarks. But as the field matures and enterprises look to draw real value from advances in AI, we\'re seeing parallel research in techniques that help productionize AI applications\n.\n\n#### **C. IPO Activity Expected**\n\n\nIt is predicted that in 2026, Anthropic will debut on public markets. It will be one of the biggest and most highly anticipated IPOs of all time\n.\n\n---\n\n### **VI. CHALLENGES & WARNINGS**\n\n\nWhile the company had the resources and investor backing to succeed, it couldn\'t retain the team that built its credibility. For the broader AI ecosystem, it\'s a reminder that even billion-dollar valuations can\'t guarantee stability when founders face the siren call of returning to the labs where they made their mark\n.\n\n\nAI agents just aren\'t generally ready for prime-time business. Various experiments by vendor and university researchers—including Anthropic and Carnegie Mellon—have found that AI agents make too many mistakes for businesses to rely on them for any process involving big money\n.\n\n---\n\n### **PRIMARY SOURCES REFERENCED**\n\n| Source | Focus |\n|--------|-------|\n| TechCrunch (Jan 24, 2026) | Original article on AI labs\' commercial ambition |\n| TechCrunch (Jan 20, 2026) | Humans& $480M seed round coverage |\n| TechCrunch (Jan 14, 2026) | Thinking Machines Lab leadership departures |\n| Bloomberg (Jan 20, 2026) | Humans& valuation and investor details |\n| Fortune (Jan 16, 2026) | Analysis of Thinking Machines departures |\n| MIT Sloan (2026) | Enterprise AI adoption challenges |\n| Radical Ventures (2026) | 10 AI predictions for 2026 |\n| VentureBeat (2026) | AI research trends for enterprise |\n\n---\n\nThis analysis shows that **2026 represents a critical inflection point**: mega-funding rounds reward pedigree, but execution on productization will determine winners from losers. The Thinking Machines Lab exodus demonstrates that talent retention and clear product strategy matter more than capital, while companies like LMArena and Converge Bio show that commercialization-focused startups can achieve traction faster than pure research labs.'}]

claude_article_builder(a)

[[{'model': 'claude',
   'headline': "AI's Reckoning: When Half-Billion Dollar Seed Rounds Meet the Hard Reality of Building Products",
   'content': 'The era of blank checks for AI pedigree is colliding with an uncomfortable truth: talent and capital don\'t guarantee execution.\n\nThe artificial intelligence industry is experiencing a dramatic identity crisis in 2026. After years of pouring billions into ever-larger language models and chasing benchmark supremacy, the sector is confronting a sobering realization—hype doesn\'t pay the bills, and customers want tools that work, not just impressive demos.\n\nAt the center of this transformation sits a peculiar paradox: AI startups are raising record-breaking funding rounds based purely on their founders\' résumés, even as one of the most well-capitalized labs is hemorrhaging the very talent that justified its $2 billion valuation.\n\n## The $480 Million Bet on Nothing\n\nHumans&, a company barely three months old with approximately 20 em

In [315]:
def json_formatter(final_output, model, event_id):

    json_format = []
    
    response_json = client.chat.completions.create(
            model = "gpt-4o-mini",
            messages = [
                {"role": "system", "content": f"""Your job is to look at the entire output of 
                {final_output} and convert into a STRICT JSON format
                
                headline is pretty straightforward to determine
                content: needs to be the full content after the headline
                sources: are the links and sources from which the article is created
                model & event id would be given in the input in the form of {model} and {event_id} respectively, so just paste that in the
                JSON format

                YOU ARE NOT SUPPOSED TO UPDATE ANYTHING ON YOUR OWN, YOU NEED TO JUST TRANSFORM THE CONTENT INTO A JSON FORMAT
                """},
                {"role": "assistant", "content": f""" 
                {{
                  "model": "{model}",
                  "headline": "<headline here>",
                  "content": "<full article content here>",
                  "sources": ["<source here>"],
                  "event_id": "{event_id}"
                }}
                """}
            ]
        )

    raw_response = response_json.choices[0].message.content
    
    clean_json = raw_response.replace("```json", "").replace("```", "").strip()

    article_data_cleaned = json.loads(clean_json)

    json_format.append(article_data_cleaned)

    return json_format